In [23]:
# 本代码由可视化策略环境自动生成 2021年8月22日11:14
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m1_run_bigquant_run(input_1, input_2, input_3):
    import re
    import requests
    import json
    df1 = pd.DataFrame()
    url = "http://fundf10.eastmoney.com/FundArchivesDatas.aspx?type=jjcc&code=004190&topline=10&year=&month=6&rt=0.6632225928306066"
    response = requests.get(url)
    data = response.text
    ins_para = re.compile(">([0-9]{6})<")
    ins_list = ins_para.findall(data)
    instrument = [str(s) + '.SHA'  if s[0] == '6' else str(s) + '.SZA' for s in ins_list]
    percent_para = re.compile("<td class='tor'>([0-9]*.[0-9]*)%</td><td class='tor'>[0-9]*.[0-9]*</td><td class='tor'>[0-9]*.[0-9]*</td>")
    percent_list = percent_para.findall(data)
    percent = [ float(s)/100 for s in percent_list ]
    print(instrument[:len(percent_list)], '\n', percent)
    buy_list=pd.DataFrame({'instrument':instrument[:len(percent_list)],
                           'buy_percent':percent},
                          index=range(len(percent)))
    print(buy_list)
    #data_1输出证券代码列表
    data_1 = {'instruments':instrument,
            'start_date':'2018-03-31',
            'end_date':'2018-06-30'}
    #data_2输出招商基金的持仓量数据
    data_2 = DataSource().write_df(buy_list)
    return Outputs(data_1=data_1, data_2=data_2, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m1_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m12_initialize_bigquant_run(context):
    # 加载仓位数据
    context.buy_list = context.options['data'].read_df()
    context.extension['ix'] = 0
    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
# 回测引擎：每日数据处理函数，每天执行一次
def m12_handle_data_bigquant_run(context, data):
    #买入并持有
    context.extension['ix'] += 1
    if  context.extension['ix'] != 1:
        return
    
    buy_list = dict(zip(context.buy_list.instrument,context.buy_list.buy_percent))
    
    for i,instrument in enumerate(buy_list):
        sid=symbol(instrument)
        order_target_percent(sid,buy_list[instrument])
        
    
# 回测引擎：准备数据，只执行一次
def m12_prepare_bigquant_run(context):
    pass
# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m12_before_trading_start_bigquant_run(context, data):
    pass

# 回测引擎：初始化函数，只执行一次
def m16_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    context.extension['ix'] = 0    
# 回测引擎：每日数据处理函数，每天执行一次
def m16_handle_data_bigquant_run(context, data):
    #买入并持有
    context.extension['ix'] += 1
    if context.extension['ix'] % 10 != 1:
            return
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]
    buy_instruments = list(ranker_prediction.instrument)[:10]
    stock_count = len(buy_instruments)
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    stock_weights = T.norm([1 / math.log(i+2) for i in range(0, stock_count)])
    max_per_instrument = 0.15
    buy_list = dict(zip(buy_instruments,stock_weights))
    for i, instrument in enumerate(buy_list):
        if buy_list[instrument] > max_per_instrument:
            context.order_target_percent(symbol(instrument), max_per_instrument)
        else:
            context.order_target_percent(symbol(instrument), buy_list[instrument])
 
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]

    # 卖出
    for stock in stock_hold_now:
        if data.can_trade(context.symbol(stock)):
            # order_target_percent是平台的一个下单接口，表明下单使得该股票的权重为0，
            #   即卖出全部股票，可参考回测文档
            context.order_target_percent(context.symbol(stock), 0)
# 回测引擎：准备数据，只执行一次
def m16_prepare_bigquant_run(context):
    pass

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m16_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.cached.v3(
    run=m1_run_bigquant_run,
    post_run=m1_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m12 = M.trade.v4(
    instruments=m1.data_1,
    options_data=m1.data_2,
    start_date='',
    end_date='',
    initialize=m12_initialize_bigquant_run,
    handle_data=m12_handle_data_bigquant_run,
    prepare=m12_prepare_bigquant_run,
    before_trading_start=m12_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)

m3 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
return_5
return_10
return_20
avg_amount_0/avg_amount_5
avg_amount_5/avg_amount_20
rank_avg_amount_0/rank_avg_amount_5
rank_avg_amount_5/rank_avg_amount_10
rank_return_0
rank_return_5
rank_return_10
rank_return_0/rank_return_5
rank_return_5/rank_return_10
pe_ttm_0""",
    m_cached=False
)

m19 = M.input_features.v1(
    features_ds=m3.data,
    features="""
# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
in_csi300_0
"""
)

m2 = M.general_feature_extractor.v7(
    instruments=m1.data_1,
    features=m19.data,
    start_date='',
    end_date='',
    before_start_days=90
)

m4 = M.derived_feature_extractor.v3(
    input_data=m2.data,
    features=m19.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m20 = M.dropnan.v2(
    input_data=m4.data
)

m8 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2018-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m6 = M.general_feature_extractor.v7(
    instruments=m8.data,
    features=m19.data,
    start_date='',
    end_date='',
    before_start_days=90
)

m7 = M.derived_feature_extractor.v3(
    input_data=m6.data,
    features=m19.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False,
    user_functions={}
)

m9 = M.advanced_auto_labeler.v2(
    instruments=m8.data,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/data_history_data.html
#   添加benchmark_前缀，可使用对应的benchmark数据
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/big_expr.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -10) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=True,
    cast_label_int=True,
    user_functions={}
)

m11 = M.join.v3(
    data1=m9.data,
    data2=m7.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m10 = M.filter.v3(
    input_data=m11.data,
    expr='in_csi300_0==1',
    output_left_data=False
)

m5 = M.dropnan.v2(
    input_data=m10.data
)

m18 = M.stock_ranker_train.v6(
    training_ds=m5.data,
    features=m3.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    data_row_fraction=1,
    ndcg_discount_base=1,
    m_lazy_run=False
)

m15 = M.stock_ranker_predict.v5(
    model=m18.model,
    data=m20.data,
    m_lazy_run=False
)

m16 = M.trade.v4(
    instruments=m1.data_1,
    options_data=m15.predictions,
    start_date='',
    end_date='',
    initialize=m16_initialize_bigquant_run,
    handle_data=m16_handle_data_bigquant_run,
    prepare=m16_prepare_bigquant_run,
    before_trading_start=m16_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2021-08-22 11:14:16.927972] INFO: moduleinvoker: cached.v3 开始运行..

[2021-08-22 11:14:16.934525] INFO: moduleinvoker: 命中缓存

[2021-08-22 11:14:16.935793] INFO: moduleinvoker: cached.v3 运行完成[0.007832s].

[2021-08-22 11:14:18.551350] INFO: moduleinvoker: backtest.v8 开始运行..

[2021-08-22 11:14:18.556864] INFO: backtest: biglearning backtest:V8.5.0

...

[2021-08-22 11:14:26.687528] INFO: Performance: Simulated 60 trading days out of 60.

[2021-08-22 11:14:26.688937] INFO: Performance: first open: 2018-04-02 09:30:00+00:00

[2021-08-22 11:14:26.690071] INFO: Performance: last close: 2018-06-29 15:00:00+00:00

[2021-08-22 11:14:27.549125] INFO: moduleinvoker: backtest.v8 运行完成[8.997788s].

[2021-08-22 11:14:27.550565] INFO: moduleinvoker: trade.v4 运行完成[10.602853s].